ใน ep นี้เราจะสมมติตัวเองเป็น Big Data Analyst ของ Starbucks ที่รับมอบหมายงาน ให้หาทำเลที่บริษัทจะก่อสร้าง ร้านกาแฟสาขาพิเศษแบบ Starbucks Reserve Roastery สาขาต่อไป เราจะวิเคราะห์ข้อมูลภูมิศาสตร์ และข้อมูลประชากร ในแต่ละ County ในรัฐ California ประเทศสหรัฐอเมริกา เพื่อหาทำเลที่มีศักยภาพ

*Starbucks Reserve Roastery คือ ร้านกาแฟ Starbucks ขนาดใหญ่พิเศษ มีสินค้าและบริการเสริมหลากหลาย กว่าสาขาปกติ เช่น โรงคั่วกาแฟ อาหารนานาชาติ ไวน์ เล้าจน์หรู คอยบริการลูกค้า*

# 0. Install

เราจะต้อง Install kaggle เพื่อ Download Dataset, geopandas เพื่อใช้ในการวิเคราะห์ข้อมูล geospatial, folium เพื่อแสดงแผ่นที่, rtree ให้สำหรับ Spatial Join (ถ้ายังไม่ได้ Install ให้ uncomment)

In [0]:
# !apt install gdal-bin python-gdal python3-gdal 
# # Install rtree - Geopandas requirment
# !apt install python3-rtree 
# # Install Geopandas
# !pip install git+git://github.com/geopandas/geopandas.git
# # Install descartes - Geopandas requirment
# !pip install descartes 

In [0]:
# ! pip install git+https://github.com/python-visualization/folium
# ! pip install kaggle --upgrade

# 1. Import Library

Import Library เพื่อใช้ในการวิเคราะห์ Spatial Analysis และ พล็อตแผนที่แบบ Interactive

In [0]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode

import folium
from folium import *
from folium.plugins import *

import os
from pathlib import Path

ประกาศฟังก์ชันในการแสดงแผนที่โดยใช้ HTML iframe แต่ Colab ไม่ Support iframe เราจะ return Map ออกไปเลย แต่มีข้อเสียคือถ้า Map ข้อมูลมากเกินไปอาจจะไม่โชว์เลย

In [0]:
from IPython.display import IFrame, HTML

def embed_map(m, file_name):    
    m.save(file_name)
    # # VM
    # return IFrame(src=file_name, width='100%', height='500px')

    # Colab
    return m

# 2. เตรียม Path สำหรับดาวน์โหลดข้อมูล

กำหนด path ของ Config File และ Dataset ว่าจะอยู่ใน Google Drive ถ้าเราใช้ Google Colab หรือ อยู่ใน HOME ถ้าเราใช้ VM ธรรมดา และกำหนด Environment Variable ไปยังโฟลเดอร์ที่เก็บ kaggle.json

ในกรณีใช้ Colab ให้ Mount Google Drive เพื่อดึง Config File มาจาก Google Drive ส่วนตัวของเรา เมื่อเรารัน Cell ด้านล่างจะมีลิงค์ปรากฎขึ้นมาให้เรา Login กด Approve แล้ว Copy Authorization Code มาใส่ในช่องด้านล่าง แล้วกด Enter

In [7]:
dataset = 'alexisbcook/geospatial-learn-course-data'

# Google Colab
config_path = Path('/content/drive')
data_path = Path('/content/datasets/')/dataset
from google.colab import drive
drive.mount(str(config_path))
os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/My Drive/.kaggle"

## VM
# config_path = Path(os.getenv("HOME"))
# data_path = config_path/"datasets"/dataset
# data_path.mkdir(parents=True, exist_ok=True)
# os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/.kaggle"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# 3. Dataset

ในเคสนี้ เราจะใช้ Dataset พิกัดภูมิศาสตร์ของร้านกาแฟ Starbucks ในรัฐแคลิฟอเนีย ประเทศสหรัฐอเมริกา

Dataset เราจะดึงจาก Kaggle วิธี Download kaggle.json ให้ดูจาก ep ที่แล้ว

เมื่อได้ kaggle.json มาแล้ว ในกรณีใช้ Google Colab ให้นำมาใส่ไว้ในโฟลเดอร์ My Drive/.kaggle ใน Google Drive ของเรา เป็น My Drive/.kaggle/kaggle.json ถ้าใช้ VM ให้ใส่ใน HOME/.kaggle/

สั่งดาวน์โหลด Dataset จาก Kaggle พร้อมทั้ง unzip ไว้ใน data_path

In [8]:
!kaggle datasets download {dataset} -p "{data_path}" --unzip

 96% 225M/233M [00:06<00:00, 38.5MB/s]
100% 233M/233M [00:06<00:00, 37.7MB/s]


โหลดข้อมูล ร้านกาแฟ Starbucks ในรัฐแคลิฟอเนีย ประเทศสหรัฐอเมริกา แล้วเปิดดู 5 ตัวอย่างแรก

In [9]:
starbucks = pd.read_csv(data_path/"starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


จากทั้งหมด จำนวน 2821 สาขา

In [10]:
len(starbucks)

2821

## 3.1 มีกี่สาขาที่ข้อมูลพิกัดขาดหายไป

ขาดไป 5 สาขา

In [12]:
print(starbucks.isnull().sum())

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


สาขาที่อยู่ในเมือง Berkeley ข้อมูลพิกัด Latitude, Longitude ขาดหายไป

In [13]:
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


# 4. Geocode

ประกาศฟังก์ชัน สำหรับเรียก geocode แล้วมาเติมให้ Feature Latitude, Longitude

In [0]:
def my_geocoder(row):
    try:
        point = geocode(row.Address, provider='nominatim', user_agent='BUA Labs/0.0.1').geometry.iloc[0]
        row['Latitude'] = point.y
        row['Longitude'] = point.x
        return row    
    except:
        return None

เราจะใช้ที่อยู่ address ไปเรียก Geocode เพื่อนำพิกัดมาใส่ใน starbucks Pandas Dataframe ที่พิกัดขาดหายไป

In [0]:
starbucks.loc[starbucks["City"]=="Berkeley"] = starbucks.loc[starbucks["City"]=="Berkeley"].apply(lambda x: my_geocoder(x), axis=1)

# 5. Plot Map

นำข้อมูลพิกัดร้านกาแฟ Starbucks ในเมือง Berkeley ด้านบน มาพล็อตบนแผนที่

In [30]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Add a marker for each Berkeley location
for idx, row in starbucks.loc[starbucks["City"]=="Berkeley"].iterrows():    
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

# Show the map
embed_map(m_2, '25f-m_2.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/11/25f-m_2.html" frameborder="0" allowfullscreen></iframe>

# 6. Merge Data

เราจะ Merge ข้อมูลร้านกาแฟ Starbucks เข้ากับ ข้อมูลขอบเขตภูมิศาสตร์ของ County ในรัฐ California (CA)

*เคาน์ตี (county) หมายถึง เขต, มณฑล, เทศมณฑล เป็นหน่วยย่อยของการปกครอง ในประเทศสหรัฐอเมริกา ขนาดเล็กกว่ารัฐ *

In [19]:
CA_counties = gpd.read_file(data_path/"CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


โหลดข้อมูล จำนวนประชากร รายได้ และอายุ

In [0]:
CA_pop = pd.read_csv(data_path/"CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv(data_path/"CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv(data_path/"CA_county_median_age.csv", index_col="GEOID")

Merge ด้วย GEOID

In [0]:
CA_stats = CA_counties.merge(CA_pop, on='GEOID').merge(CA_high_earners, on='GEOID').merge(CA_median_age, on='GEOID')

สร้าง Feature ใหม่ ชื่อ density คือ จำนวนประชากร ต่อพื้นที่ ตารางกิโลเมตร

In [22]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]
CA_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0,1.198638
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9,598.376878
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7,45.499825
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6,16.980022
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5,148.788107


# 7. Spatial Analysis

เราได้รวมข้อมูลทุกอย่างมาอยู่ใน GeoDataFrame เดียว ชื่อ `CA_stats` เพื่อให้ง่ายต่อการวิเคราะห์ คัดสรร County ที่ตรงกับเงื่อนไขความต้องการทางธุรกิจ ดังนี้

* มีอย่างน้อย 100,000 หลังคาเรือน ที่มีรายได้มากกว่า 150,000 USD ต่อปี และ
* Median อายุ น้อยกว่า 38.5 และ
* ความหนาแน่นประชากร ไม่น้อยกว่า 285 คน ต่อตารางกิโลเมตร

ยิ่งไปกว่านั้น County ที่เลือก ต้องเข้าเงื่อนไข ข้อใดข้อหนึ่ง ดังรายการด้านล่าง

* มีอย่างน้อย 500,000 หลังคาเรือน ที่มีรายได้มากกว่า 150,000 USD ต่อปี 
* Median อายุ น้อยกว่า 35.5 
* ความหนาแน่นประชากร ไม่น้อยกว่า 1,400 คน ต่อตารางกิโลเมตร

เพื่อให้เข้าใจง่าย เราจะเลือก 3 ข้อบนก่อน

In [23]:
sel1 = CA_stats.loc[(CA_stats.high_earners > 100000) & (CA_stats.median_age < 38.5) & (CA_stats.density >= 285)]
sel1

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077
38,6085,Santa Clara County,3377.487898,"POLYGON ((-122.04413 37.20050, -122.04410 37.2...",1937570,221273,37.0,573.671930
41,6001,Alameda County,2127.222169,"POLYGON ((-122.28089 37.70723, -122.28179 37.7...",1666753,145696,37.3,783.534990
47,6059,Orange County,2455.308632,"POLYGON ((-117.98911 33.58580, -117.99068 33.5...",3185968,233459,37.5,1297.583513


แล้วกรองด้วย 3 ข้อล่าง อีกที

In [24]:
sel2 = sel1.loc[(sel1.high_earners > 500000) | (sel1.median_age < 35.5) | (sel1.density >= 1400)]
sel2

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


ได้ 3 County คือ Los Angeles County, San Diego County และ San Francisco County

In [0]:
sel_counties = sel2

# 8. Big Data Analytics

ใช้ข้อมูลด้านบนมาช่วยในการวิเคราะห์ ตัดสินใจ เลือกสาขา Candidate ที่อยู่ใน County ที่เราเลือก

In [0]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

สาขาทั้งหมด ที่อยู่ใน 3 County ที่เราเลือก มีจำนวน 1043 สาขา

In [28]:
next_srr = gpd.sjoin(starbucks_gdf, sel_counties)
len(next_srr)

1043

# 9. Visualize Store Locations

Visualize แสดงผล สาขาที่เป็น Candidate บนแผนที่ เนื่องจากมีสาขาหลักพัน เราจะใช้ MarkerCluster มาช่วยให้การแสดงผล Marker จะได้ไม่รกเกินไป

In [31]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# show selected store locations
mc = MarkerCluster()
for idx, row in next_srr.iterrows():
    mc.add_child(Marker([row['Latitude'], row['Longitude']]))
m_6.add_child(mc)

# Show the map
embed_map(m_6, '25f-m_6.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/11/25f-m_6.html" frameborder="0" allowfullscreen></iframe>

# Credit

* https://www.kaggle.com/alexisbcook/manipulating-geospatial-data
* http://nominatim.org/release-docs/latest/api/Overview/
* http://geopandas.org/geocoding.html
* http://geopandas.org/mergingdata.html
* https://www.bualabs.com/archives/2762/what-is-geocode-what-is-spatial-join-geodatagrame-difference-table-join-attribute-join-geospatial-ep-5/